# Miniature/spontaneous postsynaptic currents: Part 2

In part 1 we covered how to extract the mEPSCs or sEPSCs. In this chapter we will dive into a more rigorous analysis of PSCs as well as look at how the shape of a PSC effects their integration.

In [ ]:
import json
import urllib
from collections import defaultdict

import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Spinner
from bokeh.plotting import figure
from scipy import fft, optimize, signal

output_notebook()

## Relationships between variables

In [ ]:
figure1 = figure(height=250, width=400)
figure2 = figure(height=250, width=400)
variable_dict = {
    key: final_data[key]
    for key in (
        "Est Tau (ms)",
        "Rise Time (ms)",
        "Amplitude (pA)",
        "Rise Rate (pA/ms)",
        "Fit Amplitude (pA)",
        "Fit Tau (ms)",
    )
}
variable_dict["IEI (ms)"] = final_data["IEI (ms)"] + [0] * int(
    len(final_data["Est Tau (ms)"]) - len(final_data["IEI (ms)"])
)
for key, value in variable_dict.items():
    variable_dict[key] = [[0, i] for i in value]

source1 = ColumnDataSource(variable_dict)

hist_dict = {}
for i in [
    "Est Tau (ms)",
    "Rise Time (ms)",
    "Amplitude (pA)",
    "Rise Rate (pA/ms)",
    "Fit Amplitude (pA)",
    "Fit Tau (ms)",
    "IEI (ms)",
]:
    data = final_data[i]
    min_val = min(data)
    max_val = max(data)
    padding = (max_val - min_val) * 0.1  # Add 10% padding
    grid_min = min_val - padding
    grid_max = max_val + padding
    grid_min = max(grid_min, 0)
    positions = np.linspace(grid_min, grid_max, num=124)
    kernel = stats.gaussian_kde(data)
    y = kernel(positions)
    hist_dict[f"{i}_x"] = positions
    hist_dict[f"{i}_y"] = y

hist_dict["y"] = hist_dict["Amplitude (pA)_y"]
hist_dict["x"] = hist_dict["Amplitude (pA)_x"]
source2 = ColumnDataSource(hist_dict)


mline = figure1.multi_line(
    [[i, i] for i in final_data["Timestamp (ms)"]],
    source1.data["Amplitude (pA)"],
    line_width=1,
    line_alpha=0.6,
    line_color="black",
)
line = figure2.line(x="x", y="y", source=source2, line_color="black")

menu = Select(
    title="Variables",
    value="amplitude",
    options=[
        "est_tau",
        "rise_time",
        "amplitude",
        "rise_rate",
        "iei",
        "fit_amplitude",
        "fit_tau",
    ],
)
xcheck = Checkbox(label="Log(x)")

callback = CustomJS(
    args=dict(
        source1=source1,
        source2=source2,
        mline=mline,
        line=line,
        menu=menu,
        xcheck=xcheck,
    ),
    code="""
    const x_name = `${menu.value}_x`;
    if (xcheck.active) {
        var x = source2.data[x_name].map(num => Math.log10(num));
    } else {
        var x = source2.data[x_name];
    }
    mline.data_source.data.ys = source1.data[menu.value];
    mline.data_source.change.emit();
    const y = `${menu.value}_y`;
    line.data_source.data.y = source2.data[y];
    line.data_source.data.x = x;
    line.data_source.change.emit();
""",
)

menu.js_on_change("value", callback)
xcheck.js_on_change("active", callback)

show(column(row(menu, xcheck), row(figure1, figure2)))

In [ ]:
scaled_events = events - events.max(axis=1, keepdims=True)
scaled_events /= abs(scaled_events.min(axis=1, keepdims=True))
source1 = ColumnDataSource(
    {"x": list(x), "events": list(events), "scaled_events": list(scaled_events)}
)
source2 = ColumnDataSource(
    {
        "x": np.arange(max_event_length) / 10,
        "avg_event": events.mean(axis=0),
        "avg_scaled": scaled_events.mean(axis=0),
    }
)
fig1 = figure(title="Events", height=250, width=400, output_backend="webgl")
fig1.multi_line("x", "events", source=source1, alpha=0.2, color="black")
fig1.line("x", "avg_event", source=source2, color="orange")
fig2 = figure(title="Scaled events", height=250, width=400, output_backend="webgl")
fig2.multi_line("x", "scaled_events", source=source1, alpha=0.2, color="black")
fig2.line("x", "avg_scaled", source=source2, color="orange")
show(row(fig1, fig2))